In [10]:
from groq import Groq
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [11]:
# Initialize Groq client
client = Groq(api_key='ADD_YOUR_API_KEY')

# Initialize sentence transformer for embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

legal_precedents = [
    "Case 1: Smith v. Johnson (2021) - Employers must provide reasonable accommodations for employees with disabilities under the Equal Employment Opportunity Act.",
    "Case 2: Brown v. Williams (2018) - A contract cannot be enforced if one of the parties was coerced into signing it.",
    "Case 3: Johnson v. The State (2020) - Evidence obtained without a proper warrant cannot be admitted in a criminal trial.",
    "Case 4: Davis v. TechCorp (2019) - Companies are liable for damages caused by defective products under the Consumer Protection Act.",
    "Case 5: Lee v. Thompson (2022) - Property disputes must be resolved according to land registration laws, even in the case of historical ownership claims.",
    "Case 6: Parker v. EnergyCorp (2017) - This case established the legal obligation of energy companies to provide compensation for environmental damages caused by negligence.",
    "Case 7: Reed v. National Bank (2021) - The court ruled that misleading loan terms constitute a violation of consumer protection laws, leading to contract annulment.",
    "Case 8: Wilson v. AutoMakers Inc. (2020) - The precedent for product recalls in the automotive industry, requiring manufacturers to issue recalls for known defects.",
    "Case 9: Gonzalez v. Healthcare Partners (2019) - Medical malpractice cases must show a breach of duty of care to the patient to establish liability.",
    "Case 10: Carter v. Federal Government (2018) - Government immunity does not apply in cases of constitutional violations by government agencies."
]

/Users/shashwatshahi/Desktop/Fall 2024 Assignments/CSYE 7380 Gen AI/Assignment 2/env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
# Create embeddings for the legal precedents
precedent_embeddings = model.encode(legal_precedents)

In [13]:
def get_relevant_precedent(query, embeddings, texts, top_k=2):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    top_indices = np.argsort(similarities)[-top_k:]
    return [texts[i] for i in top_indices]

def generate_response(prompt, model="llama3-8b-8192"):
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a knowledgeable legal assistant providing case law and precedents."},
            {"role": "user", "content": prompt}
        ],
        model=model,
        temperature=0.2,
        max_tokens=500
    )
    return chat_completion.choices[0].message.content

In [14]:
def rag_legal_response(query, model="llama3-8b-8192"):
    relevant_precedents = get_relevant_precedent(query, precedent_embeddings, legal_precedents)
    context = "\n".join(relevant_precedents)
    prompt = f"Based on the following legal precedents:\n{context}\n\nPlease answer the lawyer's question: {query}"
    return generate_response(prompt, model)

def non_rag_legal_response(query, model="llama3-8b-8192"):
    return generate_response(query, model)


In [16]:
# Sample legal queries
queries = [
    "Can an employer refuse to provide accommodations for disabled employees?",
    "Is evidence obtained without a warrant admissible in court?",
    "What are the legal implications of coercion in contract signing?",
    "Are companies responsible for damages caused by defective products?"
]

print("Comparing RAG and non-RAG legal precedent searcher responses:")
for query in queries:
    print(f"\nQuery: {query}")
    
    print("\nRAG Response:")
    rag_result = rag_legal_response(query)
    print(rag_result)
    print('--'*50)
    
    print("\nNon-RAG Response:")
    non_rag_result = non_rag_legal_response(query)
    print(non_rag_result)
    print("\nSimilarity:", cosine_similarity(model.encode([rag_result]), model.encode([non_rag_result]))[0][0])
    print('==='*50)


Comparing RAG and non-RAG legal precedent searcher responses:

Query: Can an employer refuse to provide accommodations for disabled employees?

RAG Response:
Based on the legal precedents provided, the answer is no, an employer cannot refuse to provide accommodations for disabled employees. According to Case 1: Smith v. Johnson (2021), employers are required to provide reasonable accommodations for employees with disabilities under the Equal Employment Opportunity Act.

This precedent sets a clear legal obligation on employers to make reasonable accommodations for disabled employees, and refusal to do so would likely be considered a violation of the law.
----------------------------------------------------------------------------------------------------

Non-RAG Response:
In the United States, the Americans with Disabilities Act (ADA) and the Rehabilitation Act of 1973 require employers to provide reasonable accommodations to qualified individuals with disabilities, unless doing so wou